# Node Types
* Word Context Nodes (WCN).
These nodes are created - 1 for 1 for every new word encountered in a text. They accumulate the aggregate context for each mention of their represented word. 


* Statement Link Node (SLN)
These nodes are created for each distinct statement such as predicate or proposition. They link together the WCN's used by the statement. By traversing these links from each SLN, a reader could recall the entirety of the statements ever heard, in no order. An SLN could contain a summary vector of the statement which created it. For multisource graphs this concept could be extended to passage and document level


* Higher Level Nodes (HLN)
Nodes without any Sensory Node State, which are not created by direct parsing of text, but are added in during the reasoning process. These nodes may be useful to store supplimentary information, separately from WCN's or SLN's. The approach for placing and/or pruning these nodes would need to be explored. It may be sufficient to simply randomly scatter them. Given a competent Node Activation Learner, it may be possible to prune randomly scattered HLN's bassed on if they are ever activated. Frequently activated HLN's could attract new HLN's to be added to the area

# Node State Types
* Sensory Node State (SNS). This comes from the environment. For WCN's this will be the the contextual word embeddings, for SLN's this will be the summary vector for the statement, and higher level nodes will not have SNS nodes. These sensory states are only updated when the model encounters new data from the environment such as a new text.


* Working Memory State (WMS). This is state which is updated during the information propagation process in the GNN. These state vectors do not technically belong to the model, and will likely be flashed after every new question. The update process which alters this state is query aware, this way the working memory can be used to store and communicate information relevant to the current task. 


* Long Term Memory State (LTMS). This state is also updated during information propagation process, however its State Update Process (SUP) is not query aware, and the vectors value is updated much more slowly than WMS. The hypothesis is that the SUP for long term memory could be trained to capture fundamental interactions between nodes, interactions which transend the context of a single query, and are more universally relevant.


* Query Agnostic State (QAS). State which is not updated as a function of a query, this to prevent query state from leaking in from the non-QAS during the reasoning process, the communication should be one way. This way the only way that QAS could aid in the reasoning process would be to enact general reasoning which is universally true. 


Both WMS and LTMS require SUP's, it will likely be benneficial if both SUP's are functions of all types of states. IE WMS would update as a function of itself, the SNS of the current node, as well as the LTMS of the current node. This way all 3 states could be used in the reasoning process.

# Stabilisation of GNN
The GNN propagates for 4 steps and then an answer is selected.
Papers noted that performance dropped after 4 steps..

This is a possible area for improvement. Ideally the more propagation steps a knowledge graph makes more sure it should be.
At least it should stabilise. 
The cause of the performance drop may be noise being introduced to nodes by far away neighbours, with irrelavant information. It may also be a form of vanishing gradient.

A number of possible avenues can be explored to prevent loss of localised graph information because of noise from neighbours:
* tweaks could be made to the forgetting process in the info propagation. It should be query aware, as well as aware of the incoming and outgoing nodes states.

* An immutable copy of the original context of each WCN before propagation can be rebroadcast every step, alongside the current state. This way the original information  in each node is not lost over time to noise, however the current state can still corelate the states of the current node with neighbouring nodes.

* Adding a learned subsystem capable of selectively inhibbiting/ boosting or disabling nodes based on their estimated relevance to solving the current task. This may help separate signal from noise in large knowledge bases with many active nodes.

A general strategy to help alleviate issues arrising from more steps of propagation would be to employ some of the strategies used by other very deep networks such as ResNet which uses residual or skip connections between layers. this would translate into skip connections over timesteps in propagation

# A dynamic propagation step count
Assuming the graph stability problem  can be addressed in part at least, 4 steps may no longer be the ideal number of propagations, as the system may bennefit from further computation and correlation. Furthermore, different questions may in fact bennefit from differing numbers of steps.

The answer decider may arbitrarily give a different answers depending on which step the propagation is stopped at. It may be the case that if a high enough level of stability can be attained, the answer decider may converge on a single answer and not change after a certain time, this then may be a good time to stop and declare a final answer.

Another possibility, which  may work even under imperfect stability, is a stopping decider which is trained to recognise when the system is confident, and when it is not. Given this, propagation could stop when the stopping decider predicts a certain level of confidence. At this point the answer decider would be used to present a final answer. 

This stopping decider should learn that confidence in the first few steps is low, as relevant information may not yet have had time to correlate. Ideally this decider would be aware of the state of the system, being able to read the active nodes, or some proxy to them, and use this state to judge whether more time is needed to come to a concensus. 

# Graph Outputs
Using HLN's it is possible to iteratively collapse the graph into single node. Scatter HLN's connected to ground nodes evenly around the ground-node-space such that there are many fewer HLN's than ground nodes, these HLN's then become the tier 1 HLN's. tier 1 HLN's can be connected to make tier 2 HLN's, again such that there are fewer tier 2 than tier 1 nodes.


continueing this node count redution as you move up their tiers, you could continue until you reached 1 node, an output node. By means of node propagation, information could flow upwards from the ground nodes, to this output node. This node could then be trained to output the answer to a speccific type/format of query. As the information passes up this heirarchy it gets dimensionally reduced since each successive tier has fewer nodes. Pairing this dimensional reduction with an attention based summarisation which is query aware, each successive layer could integrate information from larger and larger chunks of the knowledge graph, while extracting and focusing the information relevant to the query. 


Multiple output nodes could exist in the same heirarchy by branching off from eachother. Paths to different output nodes could diverge at different tiers of the heirarchy, meaning output nodes may share larger sections of their heirarchy with some output nodes than others. Output nodes which represent more similar tasks/query types could branch off later, thereby sharing more of the heirarchy.  This is similar to an idea in transfer learning whereby the less abstractly related your new goal is, the more layers you have to peel off the pretrained model.

By having multiple output nodes coming up from the ground nodes, the same knowledge base could be queried in any number of ways, with some query types sharing more nodes than others. 

# Node Activation
A node is active when it participates in the information propagation process. A simple activation method is boolean - the node either broadcasts and receives information, or it doesn't. A coninuous activation level could allow for certain nodes to be activated more than others, and even for nodes to be acivated in different ways. Nodes could contain a continuous activation number for each state type that they have, such that whenever a particular state vector gets used in the reasoning process it is multiplied by activation number. In this way the activation number would either boost or dampen or turn off the signal coming from each nodes' states

# Node Activation Learner (NAL)
For a knowledge base with many more fact than a particular task requires, it is undesirable for the entire set of the graph's nodes to activate right from the beginning. Doing so is wasteful of computational resources, and it may obscure the relationship between queries and relevant facts, making training harder to do.


A learned mechanism to activate a set of starting nodes which begin the propagation process may be helpful. If informed by the query, this selection mechanism could learn to activate specific nodes which it beleives are relevant to answering the given question. Once nodes are activated, they can activate their neighbours, possibly again employing the selector to classify the neighbouring nodes as neccesary for activation or not.


Such a selector would likely require a reinforcement learner to practice answering quetions from a knowledge base. Here the actions would be to activate or not activate each of the nodes, ideally at the granularity of SLN's to reduce the node count. The reward would be the accuracy of the question answering process.


Such a learner would likely have to be built after the system is already fully capable of answering questions. IE the system should be trained using a naive graph activation function, such as 
* one which activates all nodes indescriminately
* one which activates all WLN's of the words found in the text linked to a given question (creates dependancy on knowing which text a question's answer lies in) 
* one which activates the question words only. 

The Graph Activation Learner would be an optimisation on this naive system, but may be too difficult to learn simultaneously with the question answering process itself.


# Reasoning Explainer
Knowing which information the system used to answer its question would aid in the explainability of the model. A naive explainer may simply pluck the statements out which contain the question words and the proposed answer. A more sophisticated explainer may make use of node activation. Assuming a competent NAL, the nodes which are most active would be the most relevant to the reasoning process, and answering the question. Given these node activations, the statements which contain words represented by the most active nodes can be extracted and presented. 

Such a system would not be itself trained, as there are no lables for these explainations produced, instead it would serve only to provide feedback into the workings of the system - as an analysis tool. 

If the system was trained to be capable of converting node activity into full generated statements, such as it would be if it was adapted to be able to solve long form question answering

# Semi Supervised Learning

* Next word prediction for sentence completion. Used to train the sentence summariser function for SLN's
* blanking out entities in a sentence as a more focused fact recall. can train this function without document linking nodes such as SLN's

# Transfer Learning/ Shared Modules

transfer learning involves training a network on one task, and then retraining it on another related task. All or part of the originally trained network may be used in conjuction with additional fresh parameters/layers. If there is enough overlap in the tasks - then some of the useful knowledge gained in the first task can be applied to the new task. Once trained on the second task - if tested on the original task again, the network would likely have lost performance. 

If the data from the two tasks are spliced together during training (alternated batch by batch on the same network) the parameters shared by the tasks would be forced to learn to solve both tasks simultaneously. The resulting function represented by the shared parameters could be broken up into 3 logical components: 1,2) the parts which help solve the first and second tasks, 3) the parts which help solve both simultaneously. The more structural and logical overlap there is in the two tasks - the larger the component which solves both simultaneously would be, while components which solve them independently would get smaller. The logical separation of these pieces will be refered to as horizontal separation

To handle dimensional compatibility, transfer learning often involves adding an input and output layer to the pretrained function. In the case of simultaneous training, each task may have its own input and output layers

Traditional Transfer learning involves having 2 output models ontop of a shared weight space, 
Tasks which are logically similar but structurally differ may bennef

# Misc

* try find dataset which asks multiple questions on the same text base
* experiment with flashing current states after each question vs keeping the states throughout training
* investigate context levels.  Dalvi et al. (2018) introduces two models: PROLOCAL and PROGLOBAL for differing scope of context
* Allow for a static and working query vector. The working query vector is updated via attention during the reasoning process. In this way the working query vector could incorporate and store relavent pieces of information as they are found, and when the reasoning process is over, may be directly queried for the answer
* investigate semi-supervised learning methods such as fact recall, which can take the form of passage sentence completion. Questions may be constructed which 
* 